In [1]:
# Imports
import streamlit as st
from dataclasses import dataclass
from typing import Any, List
import datetime as datetime
import pandas as pd
import hashlib

In [2]:
class Block:

    # @TODO
    # Rename the `data` attribute to `record`, and set the data type to `Record`
    data: Any

    creator_id: int
    prev_hash: str = "0"
    timestamp: str = datetime.datetime.utcnow().strftime("%H:%M:%S")
    nonce: int = 0

    def hash_block(self):
        sha = hashlib.sha256()

        record = str(self.record).encode()
        sha.update(record)

        creator_id = str(self.creator_id).encode()
        sha.update(creator_id)

        timestamp = str(self.timestamp).encode()
        sha.update(timestamp)

        prev_hash = str(self.prev_hash).encode()
        sha.update(prev_hash)

        nonce = str(self.nonce).encode()
        sha.update(nonce)

        return sha.hexdigest()


@dataclass
class PyChain:
    chain: List[Block]
    difficulty: int = 4

    def proof_of_work(self, block):

        calculated_hash = block.hash_block()

        num_of_zeros = "0" * self.difficulty

        while not calculated_hash.startswith(num_of_zeros):

            block.nonce += 1

            calculated_hash = block.hash_block()

        print("Wining Hash", calculated_hash)
        return block

    def add_block(self, candidate_block):
        block = self.proof_of_work(candidate_block)
        self.chain += [block]

    def is_valid(self):
        block_hash = self.chain[0].hash_block()

        for block in self.chain[1:]:
            if block_hash != block.prev_hash:
                print("Blockchain is invalid!")
                return False

            block_hash = block.hash_block()

        print("Blockchain is Valid")
        return True


In [3]:
def custom_hash_function(x):
    return 0

@st.cache(allow_output_mutation=True, hash_funcs={PyChain: custom_hash_function})
def setup():
    return PyChain([Block("Genesis", 0)])



In [4]:
# @TODO:
# Delete the `input_data` variable from the Streamlit interface.
# input_data = st.text_input("Block Data")

# @TODO:
# Add an input area where you can get a value for `sender` from the user.
sender = st.text_input("Sender")

# @TODO:
# Add an input area where you can get a value for `receiver` from the user.
receiver = st.text_input("Receiver")

# @TODO:
# Add an input area where you can get a value for `amount` from the user.
amount = st.number_input("Amount", min_value=0.0)

if st.button("Add Block"):
    prev_block = pychain.chain[-1]
    prev_block_hash = prev_block.hash_block()

    # @TODO
    # Update `new_block` so that `Block` consists of an attribute named `record`
    # which is set equal to a `Record` that contains the `sender`, `receiver`,
    # and `amount` values
    new_block = Block(
        data=Record(sender=sender, receiver=receiver, amount=amount),
        creator_id=42,
        prev_hash=prev_block_hash
    )

    pychain.add_block(new_block)
    st.balloons()


2024-01-24 08:53:41.321 
  command:

    streamlit run /Users/neilbaride/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [5]:
st.markdown("## The PyChain Ledger")

# Check if pychain is defined before using it
if 'pychain' in locals() or 'pychain' in globals():
    pychain_df = pd.DataFrame(pychain.chain).astype(str)
    st.write(pychain_df)

    difficulty = st.sidebar.slider("Block Difficulty", 1, 5, 2)
    pychain.difficulty = difficulty

    st.sidebar.write("# Block Inspector")
    selected_block = st.sidebar.selectbox(
        "Which block would you like to see?", pychain.chain
    )

    st.sidebar.write(selected_block)

    if st.button("Validate Chain"):
        st.write(pychain.is_valid())
else:
    st.warning("PyChain is not defined. Please add a block first.")